In [27]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np

In [28]:
# Directorio de imágenes
base_dir = 'Images'  # Directorio base donde están los folders de clases

# Crear generadores para los conjuntos de entrenamiento, validación y prueba
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)  # 20% para validación dentro del entrenamiento
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Generador para el conjunto de entrenamiento
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(256, 256),  # Ajusta el tamaño de la imagen aquí
    batch_size=32,
    class_mode='binary',  # Para clasificación binaria
    subset='training',
    shuffle=True
)

# Generador para el conjunto de validación
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    shuffle=True
)

# Generador para el conjunto de prueba
test_generator = test_datagen.flow_from_directory(
    base_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Suele ser mejor no barajar para evaluaciones precisas
)


Found 3937 images belonging to 2 classes.
Found 983 images belonging to 2 classes.
Found 4920 images belonging to 2 classes.


In [29]:
def make_classifier_model():  # Por ejemplo, para clasificar dígitos (0-9)
    model = tf.keras.Sequential()
    
    # Primera capa convolucional
    model.add(layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=[256, 256, 3]))  # Ajustar input_shape según el tamaño y canales de tus imágenes
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # Segunda capa convolucional
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # tercera capa convolucional
    model.add(layers.Conv2D(32, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # Aplanado de la salida
    model.add(layers.Flatten())
    model.add(layers.Dense(1))  # Usamos 'sigmoid' para salida binaria

    return model

In [30]:
# Crear el modelo con el número de clases deseado
model = make_classifier_model()

# Compilación del modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',   # para etiquetas enteras (p. ej., 0, 1, 2...)
              metrics=['accuracy'])

# Mostrar un resumen del modelo
model.summary()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_12 (LeakyReLU)      │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 64, 64, 128)    │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_13 (LeakyReLU)      │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 32, 32, 32)     │       102,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_14 (LeakyReLU)      │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │        32,769 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 341,921 (1.30 MB)

 Trainable params: 341,921 (1.30 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # Ajusta el número de épocas según sea necesario
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


123/123 ━━━━━━━━━━━━━━━━━━━━ 63s 503ms/step - accuracy: 0.4495 - loss: 8.5387 - val_accuracy: 0.4427 - val_loss: 8.8846
Epoch 2/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 839us/step - accuracy: 0.2812 - loss: 11.4586 - val_accuracy: 0.2174 - val_loss: 12.4766
Epoch 3/10


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


123/123 ━━━━━━━━━━━━━━━━━━━━ 62s 497ms/step - accuracy: 0.4336 - loss: 9.0298 - val_accuracy: 0.4375 - val_loss: 8.9676
Epoch 4/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 807us/step - accuracy: 0.3750 - loss: 9.9640 - val_accuracy: 0.4348 - val_loss: 9.0109
Epoch 5/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 62s 498ms/step - accuracy: 0.4411 - loss: 8.9106 - val_accuracy: 0.4406 - val_loss: 8.9178
Epoch 6/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 788us/step - accuracy: 0.4375 - loss: 8.9676 - val_accuracy: 0.3043 - val_loss: 11.0904
Epoch 7/10


2024-10-30 18:59:23.533951: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


123/123 ━━━━━━━━━━━━━━━━━━━━ 63s 508ms/step - accuracy: 0.4505 - loss: 8.7606 - val_accuracy: 0.4375 - val_loss: 8.9676
Epoch 8/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 794us/step - accuracy: 0.3125 - loss: 10.9604 - val_accuracy: 0.4348 - val_loss: 9.0109
Epoch 9/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 61s 492ms/step - accuracy: 0.4318 - loss: 9.0578 - val_accuracy: 0.4323 - val_loss: 9.0506
Epoch 10/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 784us/step - accuracy: 0.4062 - loss: 9.4658 - val_accuracy: 0.6522 - val_loss: 5.5452


In [32]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

153/153 ━━━━━━━━━━━━━━━━━━━━ 21s 135ms/step - accuracy: 0.1156 - loss: 14.0993
Test Accuracy: 43.44%
Test Loss: 9.0164


In [34]:
# Load and preprocess the image
img_path = 'capturedImages/capture_20241030_185658.jpg'
img = image.load_img(img_path, target_size=(256, 256))  # Adjust target_size if needed
img_array = image.img_to_array(img) / 255.0  # Normalize the image
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict
prediction = model.predict(img_array)
if prediction[0] > 0.5:
    print("Prediction: Glasses")
else:
    print("Prediction: No Glasses")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Prediction: Glasses
